In [42]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import time
from sklearn.preprocessing import RobustScaler

In [43]:
model_folder = 'models-Intraday-240-1-LSTM'
result_folder = 'results-Intraday-240-1-LSTM'
for directory in [model_folder,result_folder]:
    if not os.path.exists(directory):
        os.makedirs(directory)


In [44]:
SP500_df = pd.read_csv('Data/SPXconst_2020_new_index.csv')
all_companies = list(set(SP500_df.values.flatten()))
all_companies.remove(np.nan)
SP500_df

,01/1990,02/1990,03/1990,04/1990,05/1990,06/1990,07/1990,08/1990,09/1990,10/1990,...,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A,A,A,A,A,A,A,A,A,A
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AAL,AAL,AAL,AAL,AAL,AAL,AAL,AAL,AAL,AAL
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AAP,AAP,AAP,AAP,AAP,AAP,AAP,AAP,AAP,AAP
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ABBV,ABBV,ABBV,ABBV,ABBV,ABBV,ABBV,ABBV,ABBV,ABBV
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ABC,ABC,ABC,ABC,ABC,ABC,ABC,ABC,ABC,ABC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ZBRA,ZBRA,ZBRA,ZBRA,ZBRA,ZBRA,ZBRA,ZBRA,ZBRA,ZBRA
501,ZION,ZION,ZION,ZION,ZION,ZION,ZION,ZION,ZION,ZION,...,ZION,ZION,ZION,ZION,ZION,ZION,ZION,ZION,ZION,ZION
502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ZTS,ZTS,ZTS,ZTS,ZTS,ZTS,ZTS,ZTS,ZTS,ZTS
503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
constituents = {'-'.join(col.split('/')[::-1]):set(SP500_df[col].dropna()) 
                for col in SP500_df.columns}
constituents_train = {} 
for test_year in range(1993,2016):
    months = [str(t)+'-0'+str(m) if m<10 else str(t)+'-'+str(m) 
              for t in range(test_year-3,test_year) for m in range(1,13)]
    constituents_train[test_year] = [list(constituents[m]) for m in months]
    constituents_train[test_year] = set([i for sublist in constituents_train[test_year] 
                                         for i in sublist])

In [46]:
def create_label(df_open,df_close,perc=[0.5,0.5]):
    if not np.all(df_close.iloc[:,0]==df_open.iloc[:,0]):
        print('Date Index issue')
        return
    perc = [0.]+list(np.cumsum(perc))
    label = (df_close.iloc[:,1:]/df_open.iloc[:,1:]-1).apply(
            lambda x: pd.qcut(x.rank(method='first'),perc,labels=False), axis=1)
    return label[1:]
def create_stock_data(df_open,df_close,st,m=240):
    st_data = pd.DataFrame([])
    st_data['Date'] = list(df_close['Date'])
    st_data['Name'] = [st]*len(st_data)
    daily_change = df_close[st]/df_open[st]-1
    for k in range(m)[::-1]:
        st_data['IntraR'+str(k)] = daily_change.shift(k)

    st_data['IntraR-future'] = daily_change.shift(-1)  # 将后一天赋值给当前的日期  
    st_data['label'] = list(label[st])+[np.nan] #最后一个加一个nan
    st_data['Month'] = list(df_close['Date'].str[:-3]) # 去掉后面的天，留月份
    st_data = st_data.dropna()
    
    trade_year = st_data['Month'].str[:4] # 取年份
    st_data = st_data.drop(columns=['Month'])
    st_train_data = st_data[trade_year<str(test_year)] # 交易年份小于测试年份的都是训练年份
    st_test_data = st_data[trade_year==str(test_year)] # 交易年份是测试年份的则是测试年份
    return np.array(st_train_data),np.array(st_test_data)

In [37]:
ret_done = os.listdir("results-Intraday-240-1-LSTM")
ret_done = [i[15:-4] for i in ret_done]

'1998'

In [39]:
for test_year in range(1993,2020):
    if str(test_year) not in ret_done:
        print('-'*40)
        print(test_year)
        print('-'*40)
    
#     filename = 'Open_Close/Open-'+str(test_year-3)+'.csv'
#     df_open = pd.read_csv(filename)
#     filename = 'Open_Close/Close-'+str(test_year-3)+'.csv'
#     df_close = pd.read_csv(filename)
#     colums_open = df_open.columns
#     df_open[colums_open] = df_open[colums_open].replace(0,np.nan)
#     colums_close = df_open.columns
#     df_open[colums_close] = df_open[colums_close].replace(0,np.nan)
#     label = create_label(df_open,df_close)
#     stock_names = sorted(list(constituents[str(test_year-1)+'-12']))
#     train_data,test_data = [],[]

#     start = time.time()
#     for st in stock_names:
#         st_train_data,st_test_data = create_stock_data(df_open,df_close,st)
#         train_data.append(st_train_data)
#         test_data.append(st_test_data)
        
#     train_data = np.concatenate([x for x in train_data])
#     test_data = np.concatenate([x for x in test_data])
#     # 标准化
#     scalar_normalize(train_data,test_data)
#     print(train_data.shape,test_data.shape,time.time()-start)
    
#     model,predictions = trainer(train_data,test_data)
#     # 模拟？
#     returns = simulate(test_data,predictions)
#     returns.to_csv(result_folder+'/avg_daily_rets-'+str(test_year)+'.csv')
    
#     result = Statistics(returns.sum(axis=1))
#     print('\nAverage returns prior to transaction charges')
#     result.report()
    
#     with open(result_folder+"/avg_returns.txt", "a") as myfile:
#         res = '-'*30 + '\n'
#         res += str(test_year) + '\n'
#         res += 'Mean = ' + str(result.mean()) + '\n'
#         res += 'Sharpe = '+str(result.sharpe()) + '\n'
#         res += 'std = '+str(result.std()) + '\n'
#         res += 'Standard Error = '+str(result.stderr()) + '\n'
#         res += 'Share>0 = '+str(result.pos_perc()) + '\n'
#         res += 'Skewness = '+str(result.skewness()) + '\n'
#         res += 'Kurtosis = '+str(result.kurtosis()) + '\n'
#         res += 'VaR_1 = '+str(result.VaR(1)) + '\n'
#         res += 'VaR_2 = '+str(result.VaR(2)) + '\n'
#         res += 'VaR_5 = '+str(result.VaR(5)) + '\n'
#         res += 'CVaR_1 = '+str(result.CVaR(1)) + '\n'
#         res += 'CVaR_2 = '+str(result.CVaR(2)) + '\n'
#         res += 'CVaR_5 = '+str(result.CVaR(5)) + '\n'
#         res += 'MDD = '+str(result.MDD()) + '\n'
#         myfile.write(res)

----------------------------------------
2000
----------------------------------------
----------------------------------------
2001
----------------------------------------
----------------------------------------
2002
----------------------------------------
----------------------------------------
2003
----------------------------------------
----------------------------------------
2004
----------------------------------------
----------------------------------------
2005
----------------------------------------
----------------------------------------
2006
----------------------------------------
----------------------------------------
2007
----------------------------------------
----------------------------------------
2008
----------------------------------------
----------------------------------------
2009
----------------------------------------
----------------------------------------
2010
----------------------------------------
----------------------------------------
20

In [47]:
test_year = 1993
# for test_year in range(1993,2020):
#     print('-'*40)
#     print(test_year)
#     print('-'*40)
filename = 'Open_Close/Open-'+str(test_year-3)+'.csv'
df_open = pd.read_csv(filename)
filename = 'Open_Close/Close-'+str(test_year-3)+'.csv'
df_close = pd.read_csv(filename)
colums_open = df_open.columns
df_open[colums_open] = df_open[colums_open].replace(0,np.nan)
colums_close = df_open.columns
df_open[colums_close] = df_open[colums_close].replace(0,np.nan)
df_open.loc[:,:]

,Date,AAPL,CSCO,UAL,TROW,ISRG,NVR,TPR,DVN,CE,...,CRM,PGR,WAT,IEX,BWA,LRCX,NWL,UAA,BLK,PPL
0,1990-01-02,0.251798,NaN,NaN,0.921079,NaN,4.824210,NaN,NaN,NaN,...,NaN,0.564747,NaN,2.177134,NaN,NaN,4.524198,NaN,NaN,1.844329
1,1990-01-03,0.271442,NaN,NaN,0.936560,NaN,4.944816,NaN,NaN,NaN,...,NaN,0.570284,NaN,2.209388,NaN,NaN,4.856481,NaN,NaN,1.860459
2,1990-01-04,0.273228,NaN,NaN,0.944300,NaN,4.824211,NaN,NaN,NaN,...,NaN,0.568438,NaN,2.177133,NaN,NaN,4.882043,NaN,NaN,1.849706
3,1990-01-05,0.269656,NaN,NaN,0.952040,NaN,5.065421,NaN,NaN,NaN,...,NaN,0.568438,NaN,2.112624,NaN,NaN,4.830923,NaN,NaN,1.833574
4,1990-01-08,0.267870,NaN,NaN,0.959780,NaN,4.944816,NaN,NaN,NaN,...,NaN,0.568437,NaN,2.080372,NaN,NaN,4.805360,NaN,NaN,1.812066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,1993-12-27,0.206849,1.255515,NaN,1.954947,NaN,9.250000,NaN,6.712947,NaN,...,NaN,1.785410,NaN,4.531662,2.283227,8.793470,8.962267,NaN,NaN,3.017184
1009,1993-12-28,0.214302,1.250494,NaN,1.954947,NaN,9.125000,NaN,6.930897,NaN,...,NaN,1.768187,NaN,4.547788,2.395150,8.941261,8.907456,NaN,NaN,3.017184
1010,1993-12-29,0.218030,1.265560,NaN,1.921241,NaN,9.125000,NaN,7.018077,NaN,...,NaN,1.791150,NaN,4.596169,2.395151,8.941261,8.880045,NaN,NaN,3.031089
1011,1993-12-30,0.212439,1.285648,NaN,1.954947,NaN,9.125000,NaN,7.105260,NaN,...,NaN,1.819853,NaN,4.580043,2.439920,9.015156,8.770417,NaN,NaN,3.031090


In [48]:
label = create_label(df_open,df_close)

In [49]:
stock_names = sorted(list(constituents[str(test_year-1)+'-12']))

In [50]:
train_data,test_data = [],[]
start = time.time()

In [51]:
for st in stock_names:
        st_train_data,st_test_data = create_stock_data(df_open,df_close,st)
        train_data.append(st_train_data)
        test_data.append(st_test_data)

D:\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  from ipykernel import kernelapp as app
D:\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  app.launch_new_instance()
D:\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all 

In [52]:
train_data

[array([['1990-12-11', 'ABMD', 0.04444444444444451, ...,
         0.04166666666666674, -0.020000000000000018, 0.0],
        ['1990-12-12', 'ABMD', 0.0, ..., -0.020000000000000018, 0.0, 1.0],
        ['1990-12-13', 'ABMD', -0.01098901098901095, ..., 0.0,
         -0.02083333333333337, 0.0],
        ...,
        ['1992-12-29', 'ABMD', -0.007246376811594235, ...,
         0.023255813953488413, -0.024390243902439046, 0.0],
        ['1992-12-30', 'ABMD', -0.0888888888888889, ...,
         -0.024390243902439046, 0.04878048780487809, 1.0],
        ['1992-12-31', 'ABMD', 0.008130081300812941, ...,
         0.04878048780487809, 0.0, 1.0]], dtype=object),
 array([['1990-12-11', 'ABT', 0.01465186756511172, ...,
         -0.00554013438389811, 0.019608018629978963, 1.0],
        ['1990-12-12', 'ABT', 0.0018017916493364705, ...,
         0.019608018629978963, -0.013661216395837172, 0.0],
        ['1990-12-13', 'ABT', -0.003590644113514063, ...,
         -0.013661216395837172, -0.011049567604386024, 

In [53]:
train_data = np.concatenate([x for x in train_data])
test_data = np.concatenate([x for x in test_data])

In [54]:
train_data

array([['1990-12-11', 'ABMD', 0.04444444444444451, ...,
        0.04166666666666674, -0.020000000000000018, 0.0],
       ['1990-12-12', 'ABMD', 0.0, ..., -0.020000000000000018, 0.0, 1.0],
       ['1990-12-13', 'ABMD', -0.01098901098901095, ..., 0.0,
        -0.02083333333333337, 0.0],
       ...,
       ['1992-12-29', 'ZION', 0.028735632183907844, ...,
        -0.006514657980456029, 0.019736842105263275, 1.0],
       ['1992-12-30', 'ZION', 0.00574712643678188, ...,
        0.019736842105263275, -0.019354838709677247, 0.0],
       ['1992-12-31', 'ZION', 0.0, ..., -0.019354838709677247,
        -0.02250803858520878, 0.0]], dtype=object)

In [55]:
scaler = RobustScaler()
scaler.fit(train_data[:,2:-2])

RobustScaler()